In [1]:
import cv2
import glob
import matplotlib
import numpy as np
import os
import torch

from depth_anything_v2.dpt import DepthAnythingV2

DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

depth_anything = DepthAnythingV2(**model_configs['vits'])
depth_anything.load_state_dict(torch.load(f'checkpoints/depth_anything_v2_vits.pth', map_location='cpu'))
depth_anything = depth_anything.to(DEVICE).eval()

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
xFormers not available
xFormers not available


In [2]:
sample_input = torch.rand(1, 3, 224, 224, device=DEVICE)
traced_model = torch.jit.trace(depth_anything, sample_input)

/Users/macbookpro/Desktop/project/Depth_ML/depth_anything_v2/dinov2_layers/patch_embed.py:73: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert H % patch_H == 0, f"Input image height {H} is not a multiple of patch height {patch_H}"
/Users/macbookpro/Desktop/project/Depth_ML/depth_anything_v2/dinov2_layers/patch_embed.py:74: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert W % patch_W == 0, f"Input image width {W} is not a multiple of patch width: {patch_W}"
/Users/macbookpro/Desktop/project/Depth_ML/depth_anything_v2/dinov2.py:183: TracerWarning

In [4]:
import coremltools as ct

mlmodel = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="input", shape=sample_input.shape)],
    minimum_deployment_target=ct.target.iOS15,
    convert_to="mlprogram",
    debug=True,  # Helps in getting more detailed error logs
)



Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 145.51 passes/s]


In [5]:
mlmodel.save("depth.mlpackage")